In [390]:
import pandas as pd

df = pd.read_csv("C:/Users/Mohammed/Downloads/Combined_Funda_data.csv")

In [392]:
df = df.drop(columns=["Aangeboden sinds", "Vraagprijs per m²", "Status", "Gelegen op",
                     "Zijtuin", "Bouwperiode", "Looptijd", "Toegankelijkheid", "Keurmerken",
                     "Specifiek", "Oppervlakte"])

In [396]:
# Clean the "Laatste vraagprijs" column to handle complex values like "€ 2.330.000 vrij op naam"
df['Laatste vraagprijs'] = (
    df['Laatste vraagprijs']
    .str.replace('€', '', regex=False)  # Remove the euro symbol
    .str.replace('vrij op naam', '', case=False, regex=False)  # Remove text like 'vrij op naam'
    .str.replace(r'[^\d.,]', '', regex=True)  # Keep only digits, commas, and dots
    .str.replace(',', '')  # Remove commas
    .str.replace('.', '', regex=False)  # Remove dots
    .str.strip()  # Remove leading/trailing whitespace
)

# Convert valid entries to float, filling invalid ones with NaN
df['Laatste vraagprijs'] = pd.to_numeric(df['Laatste vraagprijs'], errors='coerce')

# Fill missing values (NaN) with the mean of the column
df['Laatste vraagprijs'] = df['Laatste vraagprijs'].fillna(df['Laatste vraagprijs'].mean())

# Convert the column to integer
df['Laatste vraagprijs'] = df['Laatste vraagprijs'].astype(int)

# Verify the transformation
print(df['Laatste vraagprijs'].head())

0     485000
1     375000
2     279500
3    2330000
4    2270000
Name: Laatste vraagprijs, dtype: int32


In [398]:
# Convert the "Verkoopdatum" column to datetime, with errors as NaT
df['Verkoopdatum'] = pd.to_datetime(df['Verkoopdatum'], format='%d %B %Y', errors='coerce')

# Fill invalid or missing dates with a default value (optional)
default_date = pd.Timestamp('2023-01-01')  # Replace with a meaningful default
df['Verkoopdatum'] = df['Verkoopdatum'].fillna(default_date)

# Extract year and month as integers
df['year'] = df['Verkoopdatum'].dt.year.astype(int)
df['month'] = df['Verkoopdatum'].dt.month.astype(int)

df = df.drop(columns=["Verkoopdatum"])

In [400]:
# List of features to clean and process
features_to_clean = ["Gebouwgebonden buitenruimte", "Externe bergruimte"]

# Function to clean and convert the columns
def clean_numeric_features(df, columns):
    for column in columns:
        # Extract numbers from the column
        df[column] = df[column].str.extract(r'(\d+)')[0].astype(float)
        # Fill missing values with 0 and convert to integer
        df[column] = df[column].fillna(0).astype(int)

# Apply the cleaning function to the specified features
clean_numeric_features(df, features_to_clean)

# Verify the transformation
print(df[features_to_clean].head())

   Gebouwgebonden buitenruimte  Externe bergruimte
0                            0                   0
1                            0                   5
2                            0                   0
3                            0                   0
4                            0                   0


In [402]:
from gensim.models import Word2Vec
import numpy as np

# List of features to apply Word2Vec and process
features_to_process = [
    "Badkamervoorzieningen","Soort appartement", "Soort dak", "Balkon/dakterras", 
    "Schuur/berging", "Soort woonhuis", "Cv-ketel", "Voorzieningen", "Isolatie", "Verwarming",
    "Tuin", "Achtertuin", "Ligging", "Soort garage", "Soort parkeergelegenheid", "Warm water"
]

# Function to preprocess, train Word2Vec, and transform a column
def process_feature_with_word2vec(df, column_name):
    # Fill missing values with empty strings for Word2Vec processing
    df[column_name] = df[column_name].fillna('').astype(str)
    
    # Preprocess: Tokenize the text by splitting on spaces
    df[column_name] = df[column_name].str.lower().str.replace(',', '').str.split()
    
    # Train Word2Vec model on the column
    model = Word2Vec(sentences=df[column_name], vector_size=100, window=5, min_count=1, workers=4)
    
    # Convert each row to the scalar mean of the word vectors
    def get_scalar_mean_vector(tokens, model):
        tokens = [token for token in tokens if token in model.wv]
        if not tokens:  # If no valid tokens, return NaN
            return np.nan
        mean_vector = np.mean([model.wv[token] for token in tokens], axis=0)
        return np.mean(mean_vector)  # Scalar mean of the vector
    
    # Apply the function to transform the column
    df[column_name] = df[column_name].apply(lambda tokens: get_scalar_mean_vector(tokens, model))
    
    # Replace missing values with the column mean
    df[column_name] = df[column_name].fillna(df[column_name].mean())

# Apply the function to each feature in the list
for feature in features_to_process:
    process_feature_with_word2vec(df, feature)

# Verify the transformation
print(df[features_to_process].head())

   Badkamervoorzieningen  Soort appartement  Soort dak  Balkon/dakterras  \
0              -0.000760           0.001502   0.007291          0.000933   
1              -0.003436           0.001502   0.007291          0.000933   
2              -0.000760           0.001502   0.004810          0.000933   
3              -0.002056           0.001502  -0.001625          0.000933   
4              -0.002056           0.001502  -0.001625          0.000933   

   Schuur/berging  Soort woonhuis  Cv-ketel  Voorzieningen  Isolatie  \
0        0.003296        0.010706  0.016203       0.022643  0.004854   
1        0.004892        0.009588  0.016203       0.024611  0.011622   
2        0.003930        0.010900  0.016203       0.022751  0.004854   
3       -0.000187        0.016022  0.016203       0.022507  0.004854   
4       -0.000187        0.016022  0.016203       0.022507  0.004854   

   Verwarming      Tuin  Achtertuin   Ligging  Soort garage  \
0   -0.001241  0.003364    0.032291  0.017186  

In [404]:
# Clean the "Overige inpandige ruimte" column
df['Overige inpandige ruimte'] = df['Overige inpandige ruimte'].str.extract(r'(\d+)')[0].astype(float).fillna(0).astype(int)

In [406]:
import pandas as pd
import numpy as np

# Define the ordinal encoding for Energielabel
energy_label_mapping = {
    'G': 1,
    'F': 2,
    'E': 3,
    'D': 4,
    'C': 5,
    'B': 6,
    'A': 7,
    'A+': 8,
    'A++': 9,
    'A+++': 10,
    'A++++': 11,
    'A+++++': 12
}

# Step 1: Clean the text by removing unwanted parts like "Wat betekent dit?"
df['Energielabel'] = df['Energielabel'].str.replace(r'\s*Wat betekent dit\?.*', '', regex=True)

# Step 2: Map the cleaned data to ordinal values
df['Energielabel'] = df['Energielabel'].map(energy_label_mapping)

# Step 3: Impute missing values ("Niet beschikbaar" and NaN) with the mean of the column
mean_value = df['Energielabel'].dropna().mean()  # Calculate mean of available values
df['Energielabel'] = df['Energielabel'].fillna(mean_value).astype(int)

In [408]:
import pandas as pd
import re

# Define the ordinal mapping for "Aantal woonlagen"
ordinal_mapping = {
    '1 woonlaag': 0,
    '1 woonlaag en een zolder': 1,
    '1 woonlaag en een zolder met vliering': 2,
    '1 woonlaag en een kelder': 3,
    '1 woonlaag, een vliering en een kelder': 4,
    '1 woonlaag en een vliering': 5,
    '1 woonlaag, een zolder en een kelder': 6,
    '2 woonlagen en een zolder': 7,
    '2 woonlagen, een zolder en een kelder': 8,
    '2 woonlagen': 9,
    '2 woonlagen en een vliering': 10,
    '2 woonlagen, een vliering en een kelder': 11,
    '2 woonlagen en een kelder': 12,
    '2 woonlagen en een zolder met vliering': 13,
    '2 woonlagen, een zolder met vliering en een kelder': 14,
    '3 woonlagen': 15,
    '3 woonlagen en een vliering': 16,
    '3 woonlagen en een zolder': 17,
    '3 woonlagen en een kelder': 18,
    '3 woonlagen, een vliering en een kelder': 19,
    '3 woonlagen, een zolder en een kelder': 20,
    '4 woonlagen': 21,
    '4 woonlagen en een kelder': 22,
    '4 woonlagen, een zolder en een kelder': 23,
    '4 woonlagen en een vliering': 24,
    '5 woonlagen': 25,
    '5 woonlagen en een kelder': 26,
    '5 woonlagen en een zolder': 27,
    '5 woonlagen, een zolder en een kelder': 28,
    '5 woonlagen en een vliering': 29,
    '6 woonlagen': 30,
    '7 woonlagen': 31,
    '8 woonlagen': 32,
    '9 woonlagen': 33,
    '10 woonlagen': 34,
    '11 woonlagen': 35,
    '12 woonlagen': 36,
    '13 woonlagen': 37,
    '14 woonlagen': 38,
    '15 woonlagen': 39,
    '20 woonlagen': 40,
    '23 woonlagen': 41,
    '29 woonlagen': 42
}

# Standardize text formatting in the "Aantal woonlagen" column
df['Aantal woonlagen'] = df['Aantal woonlagen'].str.strip()

# Apply the mapping, treating unmatched values as missing and filling them with -1
df['Aantal woonlagen'] = df['Aantal woonlagen'].map(ordinal_mapping).fillna(-1).astype(int)

# Display the updated DataFrame
df[['Aantal woonlagen']].head()

,Aantal woonlagen
0,7
1,7
2,7
3,8
4,8


In [410]:
import pandas as pd

# Define the hard-coded ordinal mapping
ordinal_mapping = {
    '1 apart toilet': 0,
    '1 badkamer': 1,
    '1 badkamer en 1 apart toilet': 2,
    '1 badkamer en 2 aparte toiletten': 3,
    '1 badkamer en 3 aparte toiletten': 4,
    '2 aparte toiletten': 5,
    '2 badkamers': 6,
    '2 badkamers en 1 apart toilet': 7,
    '2 badkamers en 2 aparte toiletten': 8,
    '2 badkamers en 3 aparte toiletten': 9,
    '3 badkamers': 10,
    '3 badkamers en 1 apart toilet': 11,
    '3 badkamers en 2 aparte toiletten': 12,
    '3 badkamers en 3 aparte toiletten': 13,
    '4 badkamers en 1 apart toilet': 14,
    '4 badkamers en 2 aparte toiletten': 15,
    '4 badkamers en 3 aparte toiletten': 16,
    '5 badkamers en 3 aparte toiletten': 17
}

# Map the values in the column to the ordinal encoding, and fill missing values with -1 or another value as needed
df['Aantal badkamers'] = df['Aantal badkamers'].map(ordinal_mapping).fillna(-1).astype(int)

In [412]:
import pandas as pd
import numpy as np

# Extract the first one or two digits, convert to float, and fill missing values with the mean
df['Aantal kamers'] = df['Aantal kamers'].str.extract(r'(\d{1,2})')[0].astype(float)  # Extract one or two digits and convert to float
df['Aantal kamers'] = df['Aantal kamers'].fillna(df['Aantal kamers'].mean()).astype(int)  # Impute missing values with mean and convert to int

In [414]:
import pandas as pd
import numpy as np

# Extract numeric part, convert to float, and fill missing values with the mean
df['Wonen'] = df['Wonen'].str.extract(r'(\d+)')[0].astype(float)  # Extract numbers and convert to float
df['Wonen'] = df['Wonen'].fillna(df['Wonen'].mean()).astype(int)   # Impute missing values with mean and convert to int

In [416]:
import pandas as pd

# Map values to binary encoding and fill missing values with 0
df['Soort bouw'] = df['Soort bouw'].map({'Bestaande bouw': 0, 'Nieuwbouw': 1}).fillna(0).astype(int)

In [418]:
import pandas as pd

# Extract numeric part, replace commas with dots, convert to float, fill missing values with 0, and then convert to integer
df['Bijdrage VvE'] = df['Bijdrage VvE'].str.extract(r'(\d+[\.,]?\d*)')[0].str.replace(',', '.').astype(float).fillna(0).astype(int)

In [420]:
# Convert "Ligging tuin" to binary: 1 if not missing, 0 if missing
df['Ligging tuin'] = df['Ligging tuin'].notna().astype(int)

# Verify the transformation
print(df['Ligging tuin'].head())

0    1
1    1
2    1
3    0
4    0
Name: Ligging tuin, dtype: int32


In [422]:
# Clean the "Capaciteit" column by extracting numbers and converting to integer
df['Capaciteit'] = df['Capaciteit'].str.extract(r'(\d+)')[0].astype(float).fillna(0).astype(int)

In [424]:
import seaborn as sns
import matplotlib.pyplot as plt

# Columns of interest and the target column
columns_to_process = [
    "Inschrijving KvK",
    "Jaarlijkse vergadering",
    "Periodieke bijdrage",
    "Reservefonds aanwezig",
    "Onderhoudsplan",
    "Opstalverzekering"
]

# Perform the conversion for all specified columns
for col in columns_to_process:
    df[col] = df[col].map({"Ja": 0, "Nee": 1}).fillna(1)

def correlate_and_impute(df, col1, col2):
    for index, row in df.iterrows():
        # Impute col2 if col1 is 0 and col2 is missing
        if row[col1] == 0 and pd.isna(row[col2]):
            df.at[index, col2] = 0
        # Impute col1 if col2 is 0 and col1 is missing
        elif row[col2] == 0 and pd.isna(row[col1]):
            df.at[index, col1] = 0

# Apply to the correlated pairs
correlate_and_impute(df, "Reservefonds aanwezig", "Periodieke bijdrage")
correlate_and_impute(df, "Onderhoudsplan", "Periodieke bijdrage")
correlate_and_impute(df, "Opstalverzekering", "Periodieke bijdrage")

In [426]:
# Clean the "Inhoud" column to retain only numbers and convert to integer
df['Inhoud'] = (
    df['Inhoud']
    .str.extract(r'(\d+)')  # Extract the first sequence of digits
    [0]  # Extract the first capture group
    .astype(float)  # Convert to float to handle missing values
    .fillna(0)  # Fill missing values with 0
    .astype(int)  # Convert to integer
)

# Verify the transformation
print(df['Inhoud'].head())

0    625
1    415
2    365
3    921
4    921
Name: Inhoud, dtype: int32


In [428]:
# Clean the "Perceel" column by extracting numbers
df['Perceel'] = df['Perceel'].str.extract(r'(\d+)')[0].astype(float)

# Replace missing values in "Perceel" with corresponding values from "Wonen"
df['Perceel'] = df['Perceel'].fillna(df['Wonen'])

# Convert the column to integer data type
df['Perceel'] = df['Perceel'].astype(int)

In [430]:
# Step 1: Remove all unique values with less than 100 instances in the "Eigendomssituatie" column
value_counts = df['Eigendomssituatie'].value_counts()
to_keep = value_counts[value_counts >= 100].index
df['Eigendomssituatie'] = df['Eigendomssituatie'].where(df['Eigendomssituatie'].isin(to_keep), np.nan)

# Step 2: Fill missing values with the most frequent value
most_frequent_value = df['Eigendomssituatie'].mode()[0]
df['Eigendomssituatie'] = df['Eigendomssituatie'].fillna(most_frequent_value)

# Step 3: One-hot encode the "Eigendomssituatie" column
df = pd.get_dummies(df, columns=['Eigendomssituatie'], prefix='Eigendomssituatie')

In [432]:
# Convert one hot encoded values to 1's
df['Eigendomssituatie_Gemeentelijk eigendom belast met erfpacht'] = df['Eigendomssituatie_Gemeentelijk eigendom belast met erfpacht'].notna().astype(int)

In [434]:
df['Eigendomssituatie_Volle eigendom'] = df['Eigendomssituatie_Volle eigendom'].notna().astype(int)

In [438]:
# Save the DataFrame to a CSV file
df.to_csv('processed_dataset.csv', index=False)

In [436]:
df.isna().sum()

Laatste vraagprijs                                             0
Bijdrage VvE                                                   0
Soort appartement                                              0
Soort bouw                                                     0
Soort dak                                                      0
Wonen                                                          0
Gebouwgebonden buitenruimte                                    0
Externe bergruimte                                             0
Inhoud                                                         0
Aantal kamers                                                  0
Aantal badkamers                                               0
Badkamervoorzieningen                                          0
Aantal woonlagen                                               0
Voorzieningen                                                  0
Energielabel                                                   0
Isolatie                 